# Extraindo dados de **TODOS os BDR's** negociados

Usaremos o portal **[InvestNews](https://investnews.com.br/)** para realizar a extração de dados.
O caminho direto para os dados de nosso interesse, é [esse](https://investnews.com.br/financas/veja-a-lista-completa-dos-bdrs-disponiveis-para-pessoas-fisicas-na-b3/).

In [1]:
url = 'https://investnews.com.br/financas/veja-a-lista-completa-dos-bdrs-disponiveis-para-pessoas-fisicas-na-b3/'

Ao acessar o site, vemos que existem **duas tabelas** disponíveis, uma contendo as **BDR's das Ações** e outra com as **BDR's de ETF's**.

**Request com a Pandas**

Faremos uma tentativa inicial, da maneira mais básica possível, usando a função [read_html](https://pandas.pydata.org/docs/reference/api/pandas.read_html.html) da pandas para fazer a leitura do código fonte da página:

In [2]:
import pandas as pd

In [3]:
pd.read_html(url)

HTTPError: HTTP Error 403: Forbidden

Vemos que houve um ["erro"](https://www.pythonpool.com/urllib-error-httperror-http-error-403-forbidden/) de acesso, que na realidade é uma proibição de acesso, uma espécie de segurança impedindo o acesso de maneira artificial para evitar ataques virtuais.
Precisamos melhorar nossa apresentação ao site, ou seja, acessá-lo de maneira mais amigável, deixando claro que mesmo estando fazendo uma extração via python, eu tenho uma identidade.

**Request com a urllib**

Nesta tentativa, usaremos a biblioteca [urllib](https://docs.python.org/3/library/urllib.html), que permite adicionar parâmetros no request, tornando nosso acesso mais orgânico:

In [4]:
from urllib import request
from urllib.request import Request, urlopen

In [5]:
pedido_site = Request(url)

Fizemos o pedido de acesso, e por hora não tivemos intercorrências.

Agora, vamos informar o que desejamos fazer com esse pedido:

In [6]:
urlopen(pedido_site).read()

HTTPError: HTTP Error 403: Forbidden

Ao informar nosso desejo de ler a informação do site, tivemos a mesma negativa por bloqueio, sinal de que as catacterísticas padrão da urllib não foram suficientes.

**Novo Request com a urllib, agora com mais parâmetros de identificação**

Melhorando argumentos para acesso:

In [ ]:
pedido_site = Request(url, headers = {"User-Agent":"Mozilla/5.0"})
urlopen(pedido_site).read()

Conseguimos obter o código fonte do site, mas ainda de maneira desordenada.

Precisamos ordenar essas informações para que possamos coletar somente o que seja de nosso interesse.
Para isso faremos uma breve análise usando novamente a função read_html.

In [8]:
codigo_html = urlopen(pedido_site).read()

In [9]:
pd.read_html(codigo_html)

[                 0       1                                       2  \
 0          EMPRESA  CÓDIGO                                   SETOR   
 1               3M  MMMC34                 Indústria diversificada   
 2         AB INBEV  ABUD34                                 Bebidas   
 3          ABB LTD  A1BB34                              Industrial   
 4           ABBOTT  ABTT34                           Farmacêuticos   
 ..             ...     ...                                     ...   
 704  RAYTHEON TECH  RYTT34                              Industrial   
 705   SCHLUMBERGER  SLBG34  Equipamentos e Serviços de Combustível   
 706        TIFFANY  TIFF34                               Joalheria   
 707     US BANCORP  USBC34                  Instituição Financeira   
 708         XP INC  XPBR31                              Financeiro   
 
                   3  
 0    PAÍS DE ORIGEM  
 1               EUA  
 2           Bélgica  
 3       Switzerland  
 4               EUA  
 ..     

In [10]:
type(pd.read_html(codigo_html))

list

Vemos que existe uma ordenação dentro do site, aparentemente dividido por tabelas.
Ao pedir o tipo do elemento, nos retorna "lista".
Possivelmente seja uma lista de dataframes, onde cada elemento seja um dataframe.

In [11]:
pd.read_html(codigo_html)[0]

,0,1,2,3
0,EMPRESA,CÓDIGO,SETOR,PAÍS DE ORIGEM
1,3M,MMMC34,Indústria diversificada,EUA
2,AB INBEV,ABUD34,Bebidas,Bélgica
3,ABB LTD,A1BB34,Industrial,Switzerland
4,ABBOTT,ABTT34,Farmacêuticos,EUA
...,...,...,...,...
704,RAYTHEON TECH,RYTT34,Industrial,EUA
705,SCHLUMBERGER,SLBG34,Equipamentos e Serviços de Combustível,EUA
706,TIFFANY,TIFF34,Joalheria,EUA
707,US BANCORP,USBC34,Instituição Financeira,EUA


No elemento 0 temos as informações dos **BDR's das Ações**;

No elemento 1 temos as informações dos **BDR's de ETF's**.

Agora vamos:
*   1- **Criar um dataframe** com a elemento [0] da lista;
*   2- **Renomear índice das colunas**, usando as informações da linha 0.
*   3- **Eliminar** informações da **linha 0**.

In [12]:
df_bdr = pd.read_html(codigo_html)[0]

In [13]:
df_bdr.columns = df_bdr.iloc[0]

In [14]:
df_bdr.drop(0, axis = 0, inplace = True)

In [15]:
df_bdr.head()

,EMPRESA,CÓDIGO,SETOR,PAÍS DE ORIGEM
1,3M,MMMC34,Indústria diversificada,EUA
2,AB INBEV,ABUD34,Bebidas,Bélgica
3,ABB LTD,A1BB34,Industrial,Switzerland
4,ABBOTT,ABTT34,Farmacêuticos,EUA
5,ABBVIE,ABBV34,Medicamentos e Outros Produtos,EUA


In [17]:
df_bdr.shape

(708, 4)

Temos **708 BDR's** para trabalhar.

Uma das possibilidades seria extrair os tickers em lista, para uso posterior:

In [18]:
bdr_ticker = list(df_bdr['CÓDIGO'])
bdr_ticker

['MMMC34',
 'ABUD34',
 'A1BB34',
 'ABTT34',
 'ABBV34',
 'A1BM34',
 'ATVI34',
 'ADBE34',
 'A1AP34',
 'A1MD34',
 'A1EG34',
 'A1ES34',
 'A1FL34',
 'AIRB34',
 'A1GI34',
 'A1PD34',
 'A1KA34',
 'A1LK34',
 'A1LB34',
 'A1RE34',
 'A1LX34',
 'BABA34',
 'A1LG34',
 'A1GN34',
 'A1LL34',
 'A1EN34',
 'A1TT34',
 'A1LN34',
 'GOGL34',
 'GOGL35',
 'A1YX34',
 'MOOO34',
 'A2MC34',
 'A1CR34',
 'A1EE34',
 'A1MX34',
 'AALL34',
 'A1EP34',
 'T1OW34',
 'A1WK34',
 'A1MP34',
 'A1MB34',
 'A1ME34',
 'AMGN34',
 'A1PH34',
 'A1DI34',
 'A1UA34',
 'A1NS34',
 'A1NT34',
 'A1OS34',
 'A1ON34',
 'A1PA34',
 'A1IV34',
 'AAPL34',
 'A1MT34',
 'APTV34',
 'ARMT34',
 'A1DM34',
 'A1RG34',
 'A1NE34',
 'AWII34',
 'A1JG34',
 'A1SN34',
 'ASML34',
 'A1SU34',
 'A1ZN34',
 'T1AM34',
 'A1TM34',
 'A1UT34',
 'A1TH34',
 'ADPR34',
 'AZOI34',
 'A2VL34',
 'A1VB34',
 'A1VY34',
 'BIDU34',
 'B1KR34',
 'B1LL34',
 'B1SA34',
 'BOAC34',
 'B1CS34',
 'B1AX34',
 'B1BT34',
 'B1DX34',
 'B1GN34',
 'BBYY34',
 'BHPG34',
 'B1BL34',
 'BILB34',
 'B1IL34',
 'B2HI34',

Assim, por exemplo, conseguimos extrair dados de cotação com a [yfinance](https://pypi.org/project/yfinance/).

Por padrão, a yfinance usa ".SA" ao final de cada ticker, precisamos fazer esse incremento.

In [19]:
bdr_sa = [bdr + '.SA' for bdr in bdr_ticker]
bdr_sa

['MMMC34.SA',
 'ABUD34.SA',
 'A1BB34.SA',
 'ABTT34.SA',
 'ABBV34.SA',
 'A1BM34.SA',
 'ATVI34.SA',
 'ADBE34.SA',
 'A1AP34.SA',
 'A1MD34.SA',
 'A1EG34.SA',
 'A1ES34.SA',
 'A1FL34.SA',
 'AIRB34.SA',
 'A1GI34.SA',
 'A1PD34.SA',
 'A1KA34.SA',
 'A1LK34.SA',
 'A1LB34.SA',
 'A1RE34.SA',
 'A1LX34.SA',
 'BABA34.SA',
 'A1LG34.SA',
 'A1GN34.SA',
 'A1LL34.SA',
 'A1EN34.SA',
 'A1TT34.SA',
 'A1LN34.SA',
 'GOGL34.SA',
 'GOGL35.SA',
 'A1YX34.SA',
 'MOOO34.SA',
 'A2MC34.SA',
 'A1CR34.SA',
 'A1EE34.SA',
 'A1MX34.SA',
 'AALL34.SA',
 'A1EP34.SA',
 'T1OW34.SA',
 'A1WK34.SA',
 'A1MP34.SA',
 'A1MB34.SA',
 'A1ME34.SA',
 'AMGN34.SA',
 'A1PH34.SA',
 'A1DI34.SA',
 'A1UA34.SA',
 'A1NS34.SA',
 'A1NT34.SA',
 'A1OS34.SA',
 'A1ON34.SA',
 'A1PA34.SA',
 'A1IV34.SA',
 'AAPL34.SA',
 'A1MT34.SA',
 'APTV34.SA',
 'ARMT34.SA',
 'A1DM34.SA',
 'A1RG34.SA',
 'A1NE34.SA',
 'AWII34.SA',
 'A1JG34.SA',
 'A1SN34.SA',
 'ASML34.SA',
 'A1SU34.SA',
 'A1ZN34.SA',
 'T1AM34.SA',
 'A1TM34.SA',
 'A1UT34.SA',
 'A1TH34.SA',
 'ADPR34.SA',
 'AZOI

In [20]:
import yfinance as yf

In [21]:
bdf_fechamentos = yf.download(bdr_sa, start = '2024-01-01')['Close']
bdf_fechamentos

[*********************100%%**********************]  708 of 708 completed
ERROR:yfinance:
65 Failed downloads:
ERROR:yfinance:['A1BM34.SA', 'S2TO34.SA', 'H1ZN34.SA', 'INGG3.SA', 'X1LN34.SA', 'A1LX34.SA', 'F1RI34.SA', 'C1HI34.SA', 'S1GE34.SA', 'E1XC34MONDE.SA', 'D1RE34.SA', 'LBRN34.SA', 'A1MX34.SA', 'A1UA34.SA', 'A1MB34.SA', 'N1LS34.SA', 'L1BT35.SA', 'A1BB34.SA', 'FLTC34.SA', 'K1CS34.SA', 'E1NI34.SA', 'C1HL34.SA', 'RDSA34.SA', 'C1SU34.SA', 'SLBC34.SA', 'CAJI34.SA', 'C1ER34.SA', 'C1OU34.SA', 'FBOK34.SA', 'A1NT34.SA', 'ATVI34.SA', 'V1AR34.SA', 'C1XO34.SA', 'M1XI34.SA', 'U1NL34.SA', 'DCVY35.SA', 'A2MC34.SA', 'P1KI34.SA', 'ISBC34.SA', 'W1BK34.SA', 'E1FX34.SA', 'SBNY34.SA', 'LSXM35.SA', 'NUBR33.SA', 'P1BC34.SA', 'A2VL34.SA', 'TIFF34.SA', 'Z2EN34.SA', 'C1TX34.SA', 'LSXM34.SA', 'L1IN34.SA', 'D1IS34.SA', 'L1BT34.SA', 'RDSA35.SA', 'I1NF34.SA', 'B1BL34.SA', 'CRHP34.SA', 'F1RC34.SA', 'PTCH34.SA', 'W1RK34.SA', 'BHPG34.SA', 'TWTR34.SA', 'C1EO34.SA', 'L1FC34.SA']: Exception('%ticker%: No timezone foun

Ticker,A1AP34.SA,A1BB34.SA,A1BM34.SA,A1CR34.SA,A1DI34.SA,A1DM34.SA,A1EE34.SA,A1EG34.SA,A1EN34.SA,A1EP34.SA,...,XRXB34.SA,YUMR34.SA,Z1BH34.SA,Z1BR34.SA,Z1IO34.SA,Z1OM34.SA,Z1TO34.SA,Z1TS34.SA,Z2EN34.SA,Z2LL34.SA
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-02,19.290001,NaN,NaN,47.200001,486.0,359.000000,174.960007,28.030001,252.750000,194.800003,...,90.629997,315.329987,172.199997,43.000000,167.500000,13.47,25.420000,60.540001,NaN,23.420000
2024-01-03,18.780001,NaN,NaN,47.650002,466.5,363.600006,174.960007,27.719999,252.750000,194.800003,...,77.849998,315.329987,172.199997,43.000000,167.500000,13.21,25.420000,59.599998,NaN,23.420000
2024-01-04,18.540001,NaN,NaN,47.070000,466.5,354.000000,174.960007,27.930000,252.750000,194.800003,...,80.320000,315.329987,172.199997,41.270000,167.500000,13.18,25.420000,59.349998,NaN,22.299999
2024-01-05,18.820000,NaN,NaN,47.450001,466.5,345.000000,174.960007,27.930000,252.750000,194.800003,...,80.320000,312.959991,172.199997,41.270000,167.500000,13.01,25.420000,59.520000,NaN,22.299999
2024-01-08,18.820000,NaN,NaN,47.560001,466.5,345.000000,174.960007,27.930000,252.750000,194.800003,...,81.000000,314.649994,148.889999,41.270000,167.500000,13.53,25.420000,59.520000,NaN,22.299999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-08,23.600000,NaN,NaN,50.049999,489.0,314.029999,187.720001,32.849998,253.889999,230.550003,...,67.830002,364.679993,158.250000,52.400002,213.149994,12.46,26.190001,52.189999,NaN,18.650000
2024-05-09,24.290001,NaN,NaN,52.389999,489.0,319.040009,187.720001,33.509998,253.889999,230.550003,...,67.830002,364.679993,158.250000,52.400002,213.149994,12.63,26.190001,53.900002,NaN,18.650000
2024-05-10,24.049999,NaN,NaN,53.500000,489.0,323.799988,187.720001,34.200001,253.889999,230.550003,...,67.830002,364.679993,158.250000,52.400002,227.240005,12.70,26.190001,54.400002,NaN,18.650000
